In [8]:
import numpy as np
import h5py
import time
import copy
from random import randint
#load MNIST data
MNIST_data = h5py.File('MNISTdata.hdf5', 'r')
x_train = np.float32(MNIST_data['x_train'][:] )
y_train = np.int32(np.array(MNIST_data['y_train'][:,0]))
x_test = np.float32( MNIST_data['x_test'][:] )
y_test = np.int32( np.array( MNIST_data['y_test'][:,0] ) )
MNIST_data.close()
#Implementation of stochastic gradient descent algorithm
#number of inputs
num_inputs = 28*28
#number of outputs
num_outputs = 10
model = {}
#number of units in hidden layer
dH = 30
#Initialization parameters
model['W'] = np.random.randn(dH,num_inputs) / np.sqrt(num_inputs)
model['b1']= np.random.randn(dH) / np.sqrt(num_inputs)
model['b2'] = np.random.randn(num_outputs) / np.sqrt(num_inputs)
model['C'] = np.random.randn(num_outputs,dH) / np.sqrt(num_inputs)
model_grads = copy.deepcopy(model)

In [9]:
def softmax_function(z):
    ZZ = np.exp(z)/np.sum(np.exp(z))
    return ZZ

def ReLu(Z):
    return np.maximum(Z,0)

def ReLuPrime(Z):
    Z[Z>=0] = 1
    Z[Z<0] = 0
    return Z.astype(int)

def forward(x,y, model):
    Z = np.dot(model['W'], x) + model['b1']
    H = ReLu(Z)
    U = np.dot(model['C'],H) + model['b2']
    p = softmax_function(U)
    return p,H,Z

def backward(x,y,p,H,Z,model, model_grads):
    dU = -1.0*p
    dU[y] = dU[y] + 1.0
    # b2 gradient
    model_grads['b2'] = dU
    # C gradient
    model_grads['C'] = np.outer(dU,H)
    # delta
    delta = np.dot(model['C'].T,dU)
    # b1 gradient
    model_grads['b1'] = delta * ReLuPrime(Z)
    # W1 gradient
    model_grads['W'] = np.outer(model_grads['b1'], x)
    return model_grads

In [10]:
import time
time1 = time.time()
LR = .01
num_epochs = 20
for epochs in range(num_epochs):
    #Learning rate schedule
    if (epochs > 5):
        LR = 0.001
    if (epochs > 10):
        LR = 0.0001
    if (epochs > 15):
        LR = 0.00001
    total_correct = 0
    for n in range( len(x_train)):
        n_random = randint(0,len(x_train)-1 )
        y = y_train[n_random]
        x = x_train[n_random][:]
        p,H,Z = forward(x, y, model)
        prediction = np.argmax(p)
        if (prediction == y):
            total_correct += 1
        model_grads = backward(x,y,p,H,Z,model, model_grads)
        model['C'] = model['C'] + LR*model_grads['C']
        model['b2'] = model['b2'] + LR*model_grads['b2']
        model['b1'] = model['b1'] + LR*model_grads['b1']
        model['W'] = model['W'] + LR*model_grads['W'] # changed from + to -
    print(total_correct/np.float(len(x_train) ) )
        
time2 = time.time()
print(time2-time1)

0.9140333333333334
0.9552666666666667
0.9630833333333333
0.9696166666666667
0.9713333333333334
0.9736833333333333
0.9822333333333333
0.9840833333333333
0.9855666666666667
0.98595
0.98755
0.9872
0.9878333333333333
0.9876166666666667
0.9876333333333334
0.9876166666666667
0.9874666666666667
0.98805
0.9885666666666667
0.9878666666666667
217.10775589942932


In [11]:
######################################################
#test data
total_correct = 0
for n in range( len(x_test)):
    y = y_test[n]
    x = x_test[n][:]
    p = forward(x, y, model)[0]
    prediction = np.argmax(p)
    if (prediction == y):
        total_correct += 1
print(total_correct/np.float(len(x_test) ) )

0.9703
